In [1]:
from h3._cy.geo import haversine_vect, geo_to_h3_vect
import h3.api.numpy_int as h3i
import h3.api.basic_str as h3s

import numpy as np

def rand_hexes(N):
    lats = np.random.uniform(0, 90, N) 
    lons = np.random.uniform(0, 90, N)

    out = np.array(
        [h3.geo_to_h3(a, o, 9) for a,o in zip(lats, lons)],
        dtype = 'uint64',
    )
    
    return out

In [7]:
%%timeit
N = 1_000_000

lats = np.random.uniform(0, 90, N) 
lons = np.random.uniform(0, 90, N)
out = np.zeros(N, dtype='uint64')

21 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit
[h3s.geo_to_h3(lat, lon, 10) for lat, lon in zip(lats, lons)]

1.89 s ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
[h3i.geo_to_h3(lat, lon, 10) for lat, lon in zip(lats, lons)]

1.54 s ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
geo_to_h3_vect(lats, lons, 9, out)

795 ms ± 7.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
.795/1.89

0.4206349206349207

# Distances

- todo: can i add in parallelization?

In [8]:


a = rand_hexes(N)
b = rand_hexes(N)

out = np.zeros(N, dtype='double')

In [3]:
%%timeit
haversine_vect(a,b,out)

1.01 s ± 34.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
out

array([2334.23311099, 2351.64597086, 7594.91934881, ..., 4098.17735216,
       1359.19258281, 6917.15880453])